In [1]:
#==Langchain测试==
from langchain_openai import ChatOpenAI
import os

In [2]:
OPENAI_API_KEY = "sk-AtyfqplUXbPUnVRn7USJTnqGS6L4dznm1ilAHDMATJmJFDlO"
BASE_URL = "https://api.fe8.cn/v1"
# llm = ChatOpenAI(model="gpt-4",openai_api_key=OPENAI_API_KEY,base_url = BASE_URL)
#在环境变量中设置openAI key 
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY #在环境变量配置openai key
# os.environ["base_url"] = BASE_URL



In [3]:
llm = ChatOpenAI(model="gpt-4",base_url = BASE_URL)
rs = llm.invoke("你是谁")
print(rs)

content='我是一个人工智能助手，由OpenAI创建，用于回答问题、提供信息和帮助用户解决各种问题。我基于大量数据训练，可以在多个领域提供帮助。你有什么问题或需要帮助的吗？我会尽力为你提供支持。' response_metadata={'token_usage': {'completion_tokens': 95, 'prompt_tokens': 11, 'total_tokens': 106}, 'model_name': 'gpt-4', 'system_fingerprint': 'fp_811936bd4f', 'finish_reason': 'stop', 'logprobs': None} id='run-d38cc232-7b87-4ba8-a051-3b7e07898b7c-0' usage_metadata={'input_tokens': 11, 'output_tokens': 95, 'total_tokens': 106}


In [15]:
#多轮模型对话
from langchain.schema import(
    AIMessage,#AI回复
    HumanMessage,#系统前置设定
    SystemMessage#用户输入
)
# msg = [
#     HumanMessage(content = "你是谁？")
# ]
msg = [
    SystemMessage(content = "燕立信是一家为企业提供离岸公司服务的公司"),
    HumanMessage(content = "请问燕立信是一个什么企业")
]
rs2 = llm.invoke(msg)
print(rs2)

content='燕立信是一家为企业提供离岸公司服务的公司。该公司主要帮助客户设立和管理离岸公司，提供有关税务规划、法律合规、公司注册等方面的咨询和服务。燕立信致力于帮助企业实现国际化经营，优化企业结构，提高运营效率。' response_metadata={'token_usage': {'completion_tokens': 111, 'prompt_tokens': 42, 'total_tokens': 153}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-2342fcce-b7f5-44f4-a6e8-361796fc87b1-0' usage_metadata={'input_tokens': 42, 'output_tokens': 111, 'total_tokens': 153}


In [5]:
msg3 = [
    AIMessage(content = "我是AI小助手，有什么可以帮到您，请提问"),
#     SystemMessage(content = "燕立信是一家为企业提供离岸公司服务的公司"),
#     HumanMessage(content = "请问燕立信是一个什么企业")
]
rs3 = llm.invoke(msg3)
print(rs3)

content='您好！请告诉我您的问题，我会尽力帮您解答。' response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 27, 'total_tokens': 51}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-2fd34eef-6401-4766-8a83-bc4292a780ff-0' usage_metadata={'input_tokens': 27, 'output_tokens': 24, 'total_tokens': 51}


In [6]:
#换模型同样结构，引用模型，设定msg = [ HumanMessage(content = ""),AIMessage(content = ""),SystemMessage(content = "")], invoke(msg)


In [7]:
#使用prompt封装改进以上功能
from langchain.prompts import(
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)

temp = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template("{info}"),
        HumanMessagePromptTemplate.from_template("{query}"),
    ]
)


prompt = temp.format_messages(
    info = "燕立信的公司地址在北京市朝阳公园附近",
    query = "请问从天安门到燕立信公司怎么走"
)

rs4 = llm.invoke(prompt)
print(rs4)

content='抱歉，我无法提供具体的路线信息。不过，您可以使用地图应用（如高德地图、百度地图或谷歌地图）来查找从您的位置到燕立信公司在北京市朝阳公园附近的具体路线。只需输入您的起始位置和目的地，就可以获得详细的路线指导。' response_metadata={'token_usage': {'completion_tokens': 107, 'prompt_tokens': 45, 'total_tokens': 152}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-798c1355-f2e6-4f69-8fed-163ccf2ec112-0' usage_metadata={'input_tokens': 45, 'output_tokens': 107, 'total_tokens': 152}


In [8]:
#方法二，多轮对话用MessagePlaceholder占位，随时传递拼接某段消息


In [9]:
#==从文件加载prompt(可以用来做RAG)==
from langchain.prompts import PromptTemplate


In [34]:
#==输出结构化 OutputParser==
#==Pydantic(JSON格式输出测试)Parser
from langchain_core.pydantic_v1 import BaseModel,Field, validator
from typing import List,Dict
#定义输出结构体
class outObj(BaseModel):
    name:str = Field(description="Name")
    tel:int = Field(description="Telephone number")
        
    #手机号码验证
    @validator('tel')
    def valid_tel(cls,field):
        if len(field)!=11:
            raise ValueError("手机号码位数不对")
        return field


In [44]:
#langChain调用解析输出结构体   
from langchain.prompts import PromptTemplate,ChatPromptTemplate,HumanMessagePromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser

outParser = PydanticOutputParser(pydantic_object = outObj)#解析输出结构体

templte = '''
请提取用户输入的姓名和手机号码,输出格式如下：
{outInfo}
不要输出未提及的字段、文字及字符，也不要输出任何json格式以外的提示和符吓，如“json”、“输出”等字样

用户输入：
{query}
'''
prmpt = PromptTemplate(
    template = templte,
    input_variable = ["query"],
    #将输出parser结构体对象转字符串，填充到Prompt模板
    partial_variables = {"outInfo":outParser.get_format_instructions()}
)
# print("==outParser==")
# print(outParser.get_format_instructions())

llm = ChatOpenAI(model="gpt-4",base_url = BASE_URL)

qry = "联系人王小燕，电话：18200000001"

model_input = prmpt.format_prompt(query=qry)

rsOut = llm.invoke(model_input)

print(rsOut)
print(rsOut.content)



content='{"name": "王小燕", "tel": 18200000001}' response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 257, 'total_tokens': 277}, 'model_name': 'gpt-4', 'system_fingerprint': 'fp_811936bd4f', 'finish_reason': 'stop', 'logprobs': None} id='run-e629bbe9-aa7e-4540-8927-c06d7c4305c1-0' usage_metadata={'input_tokens': 257, 'output_tokens': 20, 'total_tokens': 277}
{"name": "王小燕", "tel": 18200000001}


In [49]:
# from langchain.output_parsers import OutputFixingParser#json格式修复

# #输出json格式修复
# new_parser = OutputFixingParser.from_llm(parser = outParser,llm=ChatOpenAI(model="gpt-4"))
# rsOut2 = '{"name": "王小燕", "tel": 18200000001}'
# rsOutNew = new_parser.parse(rsOut2)
# print("==修改json格式输出后==")
# print(rsOutNew)

In [14]:
# templateTest = PromptTemplate.from_file("testTemplate.txt")
# templateTest = PromptTemplate.from_file("aboutYanlixin.pdf")
#【PromptTemplate.from_file gbk编码报错 用with open 打开，PromptTemplate.from_template转读取】
with open("testTemplate.txt", "r", encoding="utf-8") as file:
    template2 = file.read()
# print(template2)
templateTest = PromptTemplate.from_template(template2)
print(templateTest)
print(templateTest.format(name="燕立信"))

input_variables=['name'] template='{name}公司是做什么业务的？'
燕立信公司是做什么业务的？


In [57]:
#==LLM+RAG 向量数据库检索 ==
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter#文档切割
from langchain_community.vectorstores import Chroma#Langchain封装好的向量数据库
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA#检索模块
from langchain_community.document_loaders import PyPDFLoader#PDF文件加载器


In [12]:
f = PyPDFLoader("aboutYanlixin.pdf")
pages = f.load_and_split()#按照pdf的本身页数拆分
print(len(pages))#原文件仅有一页
print(pages[0].page_content)

1
燕立信集团
企业简介
燕立信集团北京分公司是一家专注于海外公司注册的咨询服务机构，提供全球离岸注册
地平台超过60多个国家及地区，专注于离岸公司注册、开户、年审、做账审计报税、商标
品牌保护，海外融资及资产管理，海外移居优品生活，致力于打造一站式企业配套服务平台。
燕立信北京分公司曾获"北京市诚信示范企业〃荣誉。
公司团队
燕立信集团团队成员，由资深的香港执业会计师、英国特许公认会计师、公会资深成员、
财务策划师、注册资深顾问等组成，拥有丰富的离岸公司办理经验，视野兼具专业深度与全
球广度。
企业服务
燕立信集团专注于企业服务，产品覆盖企业生命全周期，包括：香港公司注册，银行开
户，香港公司年审、审计报税；马绍尔群岛公司注册，年检及经济实质申报服务；新加坡公
司注册，美国公司注册等海外公司注册，以及海外商标注册等。
燕立信集团自成立以来，一直致力于为客户提供“企业迈向海外”的一站式服务。本着
“帮助更多企业迈向全球，帮助更多国人走向世界”的服务理念，燕立信集团致力于打造成
为受行业尊重，客户信赖的专业团队。
我们仰望星空，更脚踏实地，在这个措手不及的年份里，我们看到了希望也感受到了温
暖，我们感谢每一位客户、不负每一份信任，用心分析思考全程为您护航。
企业保障
信息保密 资料保密
严格保密客户信息 售前售后，及时响应
全程服务 关键环节100%提醒
多元产品 诚信价格
提供多领域及服务支持 合法合规专业人士执行
所有注册及开户服务 价格清晰，以合理价格享品质服务


In [13]:
cuts = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap = 20,
    length_function = len,
    add_start_index = True,
)
#切割token
paragraphs = cuts.create_documents([pages[0].page_content])
for para in paragraphs:
    print(para.page_content)
    print("------------")

燕立信集团
企业简介
燕立信集团北京分公司是一家专注于海外公司注册的咨询服务机构，提供全球离岸注册
地平台超过60多个国家及地区，专注于离岸公司注册、开户、年审、做账审计报税、商标
------------
品牌保护，海外融资及资产管理，海外移居优品生活，致力于打造一站式企业配套服务平台。
燕立信北京分公司曾获"北京市诚信示范企业〃荣誉。
公司团队
------------
公司团队
燕立信集团团队成员，由资深的香港执业会计师、英国特许公认会计师、公会资深成员、
财务策划师、注册资深顾问等组成，拥有丰富的离岸公司办理经验，视野兼具专业深度与全
球广度。
企业服务
------------
球广度。
企业服务
燕立信集团专注于企业服务，产品覆盖企业生命全周期，包括：香港公司注册，银行开
户，香港公司年审、审计报税；马绍尔群岛公司注册，年检及经济实质申报服务；新加坡公
------------
司注册，美国公司注册等海外公司注册，以及海外商标注册等。
燕立信集团自成立以来，一直致力于为客户提供“企业迈向海外”的一站式服务。本着
------------
“帮助更多企业迈向全球，帮助更多国人走向世界”的服务理念，燕立信集团致力于打造成
为受行业尊重，客户信赖的专业团队。
我们仰望星空，更脚踏实地，在这个措手不及的年份里，我们看到了希望也感受到了温
------------
暖，我们感谢每一位客户、不负每一份信任，用心分析思考全程为您护航。
企业保障
信息保密 资料保密
严格保密客户信息 售前售后，及时响应
全程服务 关键环节100%提醒
多元产品 诚信价格
------------
多元产品 诚信价格
提供多领域及服务支持 合法合规专业人士执行
所有注册及开户服务 价格清晰，以合理价格享品质服务
------------


In [17]:
#向量化入库
para2 = cuts.create_documents([pages[0].page_content])
emb = OpenAIEmbeddings(base_url = BASE_URL)
db = Chroma.from_documents(para2,emb)

#检索
searchs = db.as_retriever(search_kwargs={"k":1})#指定检索1条结果
print(searchs)
rsDoc = searchs.get_relevant_documents("燕立信的服务")
print(rsDoc[0].page_content)

tags=['Chroma', 'OpenAIEmbeddings'] vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001EE7AC24070> search_kwargs={'k': 1}
品牌保护，海外融资及资产管理，海外移居优品生活，致力于打造一站式企业配套服务平台。
燕立信北京分公司曾获"北京市诚信示范企业〃荣誉。
公司团队


In [49]:
#多轮记忆ConversationTokenBufferMemory,save_context()
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory, ConversationTokenBufferMemory

# history = ConversationBufferMemory()
#设置上下文保存条目
history = ConversationBufferWindowMemory(k=3)
#按token长度保留下下文，不会暴模型
# history = ConversationTokenBufferMemory(
#     llm = ChatOpenAI(model="gpt-4",base_url = BASE_URL),
#     max_token_limit=100)

history.save_context({"input":"燕立信做哪些境外公司注册"},{"output":"香港、美国、英国"})
print(history.load_memory_variables({}))
history.save_context({"input":"香港要准备什么材料"},{"output":"企业资质、法人身份证、企业财报"})
print(history.load_memory_variables({}))
print(history.load_memory_variables({})['history'])

{'history': 'Human: 燕立信做哪些境外公司注册\nAI: 香港、美国、英国'}
{'history': 'Human: 燕立信做哪些境外公司注册\nAI: 香港、美国、英国\nHuman: 香港要准备什么材料\nAI: 企业资质、法人身份证、企业财报'}
Human: 燕立信做哪些境外公司注册
AI: 香港、美国、英国
Human: 香港要准备什么材料
AI: 企业资质、法人身份证、企业财报


In [23]:
llmM = ChatOpenAI(model="gpt-4",base_url = BASE_URL)

In [40]:
from langchain.schema import(
    AIMessage,#AI回复
    HumanMessage,#系统前置设定
    SystemMessage#用户输入
)
# msg = [
#     HumanMessage(content = "你是谁？")
# ]
msgM = [
    HumanMessage(content = history.load_memory_variables({})['history']+"美国呢？")
]
rsM = llmM.invoke(msgM)
print(rsM)

content='AI: 美国的注册需要的材料可能包括：公司名称、公司地址、公司经营范围、股东和董事信息、注册资本、法人身份证等。具体材料可能因州和城市的法规有所不同，建议咨询专业的公司注册服务。' response_metadata={'token_usage': {'completion_tokens': 86, 'prompt_tokens': 82, 'total_tokens': 168}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-7fa1346d-6749-4ce9-b8f4-69164a45d4f9-0' usage_metadata={'input_tokens': 82, 'output_tokens': 86, 'total_tokens': 168}


In [ ]:
#上下文相关其它
#对上下文做摘要ConversationSummaryMemory
#保存Token数限制上下文，更早的做摘要ConversationSummaryBufferMemory
#将Memory存在向量数据库，用户检索到相关的返回使用VectorStoreRetrieverMemory

In [55]:
#》》》Chain链式调用 RunnablePassthrough
#例：解析一个用户的问话为结构化输出，使用LCEL链式调用来实现。
# 调用语法：定义调用流程-》运行
# run = ({"query":RunnablePassthrough()}|prompt|llm|parser)#将prompt,模型，结果，链式调用组装起来
# run.invoke("请问。。。")
# RunnablePassthrough是占位符，用户输入的信息，用它填prompt->用prompt调Llm->用llm输出pars

from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from typing import List, Dict, Optional
from enum import Enum

#定义输出结构体
class outObj(BaseModel):
#     name:str = Field(description="Name")
    name:Optional[str] = Field(description="Name",default=None)#Optional允许字段为空
    tel:int = Field(description="Telephone number")
        
    #手机号码验证
#     @validator('tel')
#     def valid_tel(cls,field):
#         if len(field)!=11:
#             raise ValueError("手机号码位数不对")
#         return field

outParser2 = PydanticOutputParser(pydantic_object=outObj)
templte = '''
请提取用户输入的姓名和手机号码,输出格式如下：
{outInfo}
不要输出未提及的字段、文字及字符，也不要输出任何json格式以外的提示和符吓，如“json”、“输出”等字样

用户输入：
{query}
'''
prompt = ChatPromptTemplate.from_messages(
    [
        ("system",templte),
        ("human","{query}"),
    ]
).partial(outInfo=outParser2.get_format_instructions())
# llmModel = ChatOpenAI(model="gpt-4-0125-preview",temperature=0)
llm = ChatOpenAI(model="gpt-4",base_url = BASE_URL)
#定义LCEL(chain链式调用)流
runChain = ({"query":RunnablePassthrough()}|prompt|llm|outParser2)
# RunnablePassthrough是占位符，用户输入的信息，用它填prompt
# ->用prompt调Llm
# ->用llm输出pars

query = "联系人王小燕，电话：18200000001"
rsChain = runChain.invoke(query)
print(rsChain)
print(rsChain.json())

name='王小燕' tel=18200000001
{"name": "\u738b\u5c0f\u71d5", "tel": 18200000001}


In [5]:
#==【完整可用】【LCEL链式调用流改写LLM+RAG结】==
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter#文档切割
from langchain_community.vectorstores import Chroma#Langchain封装好的向量数据库
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA#检索模块
from langchain_community.document_loaders import PyPDFLoader#PDF文件加载器
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

OPENAI_API_KEY = "sk-AtyfqplUXbPUnVRn7USJTnqGS6L4dznm1ilAHDMATJmJFDlO"
BASE_URL = "https://api.fe8.cn/v1"
# llm = ChatOpenAI(model="gpt-4",openai_api_key=OPENAI_API_KEY,base_url = BASE_URL)
#在环境变量中设置openAI key 
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY #在环境变量配置openai key

#load文件
f = PyPDFLoader("aboutYanlixin.pdf")
pages = f.load_and_split()#按照pdf的本身页数拆分
#定义token切割对象
cuts = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap = 20,
    length_function = len,
    add_start_index = True,
)
#切割token
tokens = cuts.create_documents([pages[0].page_content])

#向量化入库
emb = OpenAIEmbeddings(base_url = BASE_URL)
db = Chroma.from_documents(tokens,emb)

#检索
searchs = db.as_retriever(search_kwargs={"k":1})#指定检索1条结果
# print(searchs)
#输入query，查看检索结果相关内容
rsDoc = searchs.get_relevant_documents("燕立信的服务")
print(rsDoc[0].page_content)



品牌保护，海外融资及资产管理，海外移居优品生活，致力于打造一站式企业配套服务平台。
燕立信北京分公司曾获"北京市诚信示范企业〃荣誉。
公司团队


In [7]:
#定义模型
llm = ChatOpenAI(model="gpt-4",base_url = BASE_URL)

#将上文的searchs,拼接到prompt中，送入LLM
temp = "请根据已知信息回答问题：context:{info}，query:{q}"
prompt = ChatPromptTemplate.from_template(temp)

rag_chain = (
    {"info":searchs,"q":RunnablePassthrough()}
    |prompt
    |llm
    |StrOutputParser()
)
#RunnablePassthrough占位符
#StrOutputParser()输出默认字符串结构
rag_chain.invoke("燕立信的服务")

'燕立信的服务包括品牌保护，海外融资及资产管理，海外移居优品生活，以及致力于打造一站式企业配套服务平台。'

In [5]:
#==回调函数function calling==
from langchain_core.tools import tool
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import JsonOutputToolsParser

@tool
def add(v1:int,v2:int)->int:
    "两个整数相加"
    return v1+v2
#@tool+文档注释，把函数自动转为openAI的语法体系
@tool
def multiply(v1:int,v2:int)->int:
    "两个整数相乘"
    return v1*v2
#注意上面的“两个数相乘”“两个数相加”不是注解，是给大模型看的，要写清楚，用""双引号，不要用#

#调用地图软件
# @tool
# def fn3():
#     return "燕立信的精确位置在朝阳区……，从天安门到。。。需。。。"
tools = [add,multiply]
llm_with_tools = llm.bind_tools(tools)|{
    "functions":JsonOutputToolsParser(),
    "text":StrOutputParser()
}
#functions有function call 走此分支 返回json结果
#text无function call 走此分支，返回文本结果
rsTools = llm_with_tools.invoke("18的2倍是多少？请计算结果")
# rsTools = llm_with_tools.invoke("你是谁")
print(rsTools)

{'functions': [{'args': {'v1': 18, 'v2': 2}, 'type': 'multiply'}], 'text': ''}


In [7]:
#解析并执行function calling 并返回结果
from langchain_core.runnables import(RunnableMap,RunnableLambda,Runnable,RunnablePassthrough)
from operator import itemgetter
from typing import Union
import json

#==map遍历function 工具执行==
#名称到函数名映射
tool_map = {tool.name:tool for tool in tools}
print("==tool_map==")
print(tool_map)
def call_tool(tool_act:dict)->Union[str,Runnable]:#返回结果可以是str,Runnable任一类型之一
    "function call tool 执行"
    tool = tool_map[tool_act["type"]]
    return RunnablePassthrough.assign(
        output = itemgetter("args")|tool
    )
#itemgetter遍历获取字典中键为 "args" 的值
call_tool_list = RunnableLambda(call_tool).map()

#==判断函数分支==
def routeFn(rs):
    if len(rs["functions"])>0:
        return rs["functions"]
    else:
        return rs["text"]
#==调用并执行function calling==
llm_with_tools2 = llm.bind_tools(tools)|{
    "functions":JsonOutputToolsParser()|call_tool_list,
    "text":StrOutputParser()
}|RunnableLambda(routeFn)

rsTools2 = llm_with_tools2.invoke("18的2倍是多少？")
print(rsTools2)

==tool_map==
{'add': StructuredTool(name='add', description='两个整数相加', args_schema=<class 'pydantic.main.addSchema'>, func=<function add at 0x0000019975F30F70>), 'multiply': StructuredTool(name='multiply', description='两个整数相乘', args_schema=<class 'pydantic.main.multiplySchema'>, func=<function multiply at 0x0000019905EA70D0>)}
[{'args': {'v1': 18, 'v2': 2}, 'type': 'multiply', 'output': 36}]


In [8]:
#==RAG+LLM【方法总结：
#读文档》切割》向量化入库》检索》链式拼合，把检索结果拼到chain中做已知信息上下文》链接调用大模型、prompt及输出格式处理函数》返回结果
#中间插入tools回调函数分支选择】

In [3]:
#==Agent智能体==
#原理：模型自主思考选择工具，完成一个多步的复杂任务，
#核心模块有两块：Tools工具集，planning用prompt定义计划、控制步骤、分支、完成思考、选择tool执行任务。
#另外有长短时记忆模块控制上下文和action控制执行操作
#测试：实现一个Agent功能，“特朗谱最近一次被枪击是星期几”
#需要两个工具：互联网时时消息搜索工具、日期到星期转换工具
from langchain_openai import ChatOpenAI
from langchain_community.utilities import SerpAPIWrapper
from langchain.tools import Tool,tool
import calendar
import dateutil.parser as parser
from datetime import date
from datetime import datetime
from langchain.agents import AgentExecutor, create_react_agent

In [4]:
#【可以自己编写智能Prompt流程，智能体的思考能力由prompt决定】
#下载一个现有prompt react有关
from langchain import hub
import json
promptAg1 = hub.pull("hwchase17/react")
print(promptAg1.template)#查看一个智能体的prompt

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [5]:
#下载另一个现有prompt SelfAskWithSearch 自动追问有关
promptAg2 = hub.pull("hwchase17/self-ask-with-search")
print(promptAg2.template)#查看这个智能体prompt结构

Question: Who lived longer, Muhammad Ali or Alan Turing?
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali

Question: When was the founder of craigslist born?
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952

Question: Who was the maternal grandfather of George Washington?
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washin

In [ ]:
#Google SerpAPIWrapper需注册取得key，如使用百度搜索API，自己封装一个Wrapper 函数使用
#百度搜索API函数封装
# class BaiduSearchAPIWrapper:  
#     def __init__(self, api_key: str, secret_key: str):  
#         self.api_key = api_key  
#         self.secret_key = secret_key  
#         self.base_url = "https://aip.baidubce.com/rest/2.0/solution/v1/public/search"
#         #百度搜索API
  
#     def run(self, query: str) -> dict:  
#         headers = {  
#             'Content-Type': 'application/json',  
#             'Api-Key': self.api_key,  
#             'Secret-Key': self.secret_key  
#         }  
#         data = {  
#             "q": query,  
#             "from": 0,  # 起始位置  
#             "size": 10  # 返回结果条数  
#         }  
#         response = requests.post(self.base_url, json=data, headers=headers)  
#         response.raise_for_status()  # 如果响应状态码不是 200, 则引发HTTPError异常  
#         return response.json()  

In [6]:
#时时搜索工具
# search = SerpAPIWrapper()
toolsAg = []
# toolsAg = [
#     Tool.from_function(
#         func = search.run,
#         name = "时时搜索",
#         description="用于在互联网时时搜索最新的信息",
#     ),
# ]
#日期转星期工具
@tool("weekDayFn")
def weekDayFn(dateV:str)->str:
    "将日期转为星期"
    if "年" in dateV:
        dateV = datetime.strptime(dateV, '%Y年%m月%d日').strftime('%Y-%m-%d')
    dObj = parser.parse(dateV)
    return calendar.day_name[dObj.weekday()]

toolsAg += [weekDayFn]

In [10]:
weekDayFn("2024-07-13")

'Saturday'

In [33]:
#定义模型
llm = ChatOpenAI(model="gpt-4",base_url = BASE_URL)
#定义agent
agent1 = create_react_agent(llm,toolsAg,promptAg1)
#定义agent执行器
exe1 = AgentExecutor(agent=agent1,tools = toolsAg,verbose=True)
exe1.invoke({"input":"特朗普最近一次被枪击是星期几？"})



> Entering new AgentExecutor chain...
I do not have real-time or recent data about events including any incidents involving public figures such as Donald Trump. My knowledge is up to 2023, and I must rely on that information. However, I can use the weekDayFn to determine the day of the week for a given date if that date is provided.

Thought: Since I do not have information on recent events, I cannot provide the day of the week for an event without a specific date.

Final Answer: I do not have the information required to answer the question about the most recent date on which Donald Trump was shot, assuming such an event has taken place. If you provide a specific date, I could use the weekDayFn to determine the day of the week.

> Finished chain.


{'input': '特朗普最近一次被枪击是星期几？',
 'output': 'I do not have the information required to answer the question about the most recent date on which Donald Trump was shot, assuming such an event has taken place. If you provide a specific date, I could use the weekDayFn to determine the day of the week.'}

In [7]:
exe1.invoke({"input":"特朗普当选总统是星期几？"})



> Entering new AgentExecutor chain...
我需要知道特朗普当选总统的具体日期，然后我可以使用weekDayFn函数来找出那一天是星期几。
Action: Google Search
Action Input: "特朗普当选总统的日期"
Google Search is not a valid tool, try one of [weekDayFn].Thought: 由于我不能使用Google Search，我需要记得特朗普当选总统的日期。特朗普是在2016年11月8日当选为美国总统的。现在我可以使用weekDayFn来确定那天是星期几。
Action: weekDayFn
Action Input: 2016-11-08
TuesdayI now know the final answer
Final Answer: 特朗普当选总统的那天是星期二。

> Finished chain.


{'input': '特朗普当选总统是星期几？', 'output': '特朗普当选总统的那天是星期二。'}

In [15]:
# 填加时时搜索tool，爬虫版
import pandas as pd
from bs4 import BeautifulSoup as BS
import requests as req
import re
import time

#时时搜索工具
@tool("searchCurFn")
def searchCurFn(keyWord):
    "时时搜索,用于在互联网时时搜索最新的信息"
    #请求搜索列表页
    keyWord = str(keyWord)
    url1 = 'https://www.baidu.com/s?ie=utf-8&wd='+keyWord
    headers ={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36"}
    res = req.get(url1,headers=headers)

    rsHtml = res.content.decode('utf-8')#请求二进制内容转utf8，防中文乱码
    time.sleep(1)
    #print(rsHtml)
    bsObj = BS(rsHtml,'html.parser')
    leftObj = bsObj.find("div",id="content_left")
    #收集相关搜索结果集
    topList = []
    topList = leftObj.find_all("div", class_="result-op")#头部搜索结果列表
    topList += leftObj.find_all("div", class_="result")#一般搜索结果列表
    #拼接搜索结果文本
    searchListText = ""
    if len(topList)>0:
        for item in topList:
            searchListText += item.text
    print("时时搜索文本长度：",len(searchListText))
    time.sleep(1)
#     searchListText = "当地时间2024年7月13日18时15分许，美国前总统特朗普在宾州举行竞选集会发表演讲时，一名枪手从集会现场外高处向特朗普所在的演讲台“开了数枪”，子弹击中特朗普的右耳。其余的子弹伤及三名无辜群众，造成1人死亡2人重伤。特朗普满脸是血，随后立即撤离了演...详情"
    return searchListText

searchCurFn("特朗普最近一次被枪击是星期几？")

时时搜索文本长度： 1684


'\n正在思考\n\n7·13美国总统选举枪击事件(2024年7月13日... - 百度百科\ue610\n\ue66a当地时间2024年7月13日18时15分许，美国前总统特朗普在宾州举行竞选集会发表演讲时，一名枪手从集会现场外高处向特朗普所在的演讲台“开了数枪”，子弹击中特朗普的右耳。其余的子弹伤及三名无辜群众，造成1人死亡2人重伤。特朗普满脸是血，随后立即撤离了演...详情\ue734\n事件经过事件伤亡事件处置事件回应\n\n百度百科\n\n特朗普最近一次被枪击是星期几?的最新相关信息枪击特朗普嫌疑人信息公布枪击特朗普嫌疑人信息公布 美国前总统特朗普于当地时间13日傍晚在宾夕法尼亚州举行的竞选集会上遭未遂刺杀,嫌疑人在竞选集会现...新浪7月15日\n\n\ue680\n播报\n\n\n\ue67d\n暂停\n\n特朗普遭刺杀,枪声进一步撕裂了美国|新京报专栏另据《华盛顿邮报》援引巴特勒市政府消息，目前有1名枪击嫌疑人已经死亡，另有1名集会参与者死亡，两人重伤。特朗普发表声明称...新京报评论7月14日\n\n\ue680\n播报\n\n\n\ue67d\n暂停\n\n特朗普的竞选搭档万斯:曾公开辱骂特朗普,支持对中国...在当地时间13日遭遇枪击的特朗普，最近可谓正处于风口浪尖。虽然经历了一场惊心动魄的暗杀，但特朗普也因此收获了不少的关注，...扶苏梦史7小时前\n\n\ue680\n播报\n\n\n\ue67d\n暂停\n\n查看更多\ue613\n\n大家还在搜特朗普向佐治亚州州长“开火”热2024特朗普基本赢不了特朗普最新讲话视频特朗普发动战争特朗普被弹原因特朗普在位期间有没有发动战争特朗普是什么人特朗普能活多久\n\n进展:特朗普集会上遭遇枪击事件,可能是暗杀企图 - 哔哩哔哩2024年7月14日  2024年7月13号晚上9点42分美国东部时间 周六在宾夕法尼亚州的特朗普集会上爆发了枪击事件。特勤局在特朗普集会事件中迅速做出了回应。前总统特朗普在遭遇暴力...哔哩哔哩\ue62b\n\n\ue680\n播报\n\n\n\ue67d\n暂停\n\n\n\n对特朗普集会枪击事件,负全部责任!他,承认了!3天前 央视记者当地时间8月2日获悉,美国特勤局代理局长罗威在当日的新闻发布会上道歉,并承认特勤局对7月13日发生的特朗普集会枪击事件负全部责任。 罗

In [16]:
toolsAg += [searchCurFn]

#定义模型
llm = ChatOpenAI(model="gpt-4",base_url = BASE_URL)
#定义agent
agent2 = create_react_agent(llm,toolsAg,promptAg1)
#定义agent执行器
exe2 = AgentExecutor(agent=agent2,tools = toolsAg,verbose=True)
exe2.invoke({"input":"特朗普最近一次被枪击是星期几？"})



> Entering new AgentExecutor chain...
我需要找到特朗普最近一次被枪击的日期，然后用这个日期来确定是星期几。
Action: searchCurFn
Action Input: 特朗普 最近 一次 被枪击 日期
时时搜索文本长度： 1656

大家还在搜特朗普访华时间特朗普向佐治亚州州长“开火”热2024特朗普基本赢不了特朗普下月访华特朗普最新讲话视频特朗普什么时候到期特朗普什么时候下去特朗普发动战争

对特朗普集会枪击事件,负全部责任!他,承认了!4天前 央视记者当地时间8月2日获悉,美国特勤局代理局长罗威在当日的新闻发布会上道歉,并承认特勤局对7月13日发生的特朗普集会枪击事件负全部责任。 罗威表示,将全力配合美国会...微信公众平台


播报



暂停



持续更新丨特朗普集会枪击事件枪手确认为20岁宾州男子








2024年7月14日  特朗普枪击事件被定性为暗杀未遂 当地时间7月13日,美国执法官员表示,前总统特朗普集会枪击事件被定性为“暗杀未遂”,已作为刺杀案调查处理。据多名执法部门的消息人士称,枪手当时在...网易


播报



暂停





特朗普险遭爆头过程复盘:凶手200米外连射8枪,被击毙画面曝光








2024年7月27日  作为一位很有希望再次问鼎白宫的美国总统候选人,居然在大庭广众之下遭到枪击,还差一点儿就被爆头,这种场面堪与1963年的肯尼迪遇刺相比。现在越来越多的视频和照片已经流出,特朗普在...网易


播报



暂停





刚刚特朗普已出院,枪击细节披露!








2024年7月14日  据央视新闻消息,当地时间13日,美国前总统特朗普竞选团队在集会枪击事件发生后发表声明称,特朗普目前状况稳定,仍将继续出席当地时间15日在威斯康星州密尔沃基举行的共和党党代会。 ...微信公众平台


播报



暂停





突发!特朗普竞选集会现场发生枪击事件,最新情况汇总→








当地时间7月13日下午，特朗普在美国宾夕法尼亚州举行竞选集会发表演讲时，现场响起枪声。他在特勤人员保护下立即撤离了演讲台。宾州巴特勒当地检察官证实，嫌疑人已被击毙，另有集会参与者中枪身...




中国银行保险报


播报



暂停





{'input': '特朗普最近一次被枪击是星期几？', 'output': '特朗普最近一次被枪击是星期六。'}